# 📝 Reddit 규칙 위반 탐지 모델 추론

이 노트북은 사전에 훈련된 모델을 불러와 `test.csv` 데이터에 대한 예측을 수행하고, `submission.csv` 파일을 생성합니다.

### **✅ 실행 전 확인 사항**
1. **인터넷(Internet) OFF**: 노트북 설정에서 인터넷이 꺼져있는지 확인하세요.
2. **데이터셋 추가**:
    - **모델 데이터셋**: `juhwancom/reddit-violation-model-v1`
    - **원본 데이터셋**: `test.csv`가 포함된 대회 데이터

In [ ]:
import os
import pickle
import joblib
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
from sentence_transformers import CrossEncoder

print("✅ 라이브러리 임포트 완료")

## 1단계: 모델 및 전처리 도구 불러오기

캐글 데이터셋으로 업로드한 모델과 전처리에 사용했던 `scaler`, 함수 등을 불러옵니다.

In [ ]:
# 📂 --- 경로 설정 ---
# ❗❗ 'your-original-competition-data' 부분은 test.csv가 있는 실제 데이터셋 이름으로 바꿔주세요.
MODEL_DATASET_PATH = '/kaggle/input/reddit-violation-model-v1/model_output/'
TEST_CSV_PATH = '/kaggle/input/your-original-competition-data/test.csv'

# 모델 및 전처리 도구 경로
MODEL_PATH = os.path.join(MODEL_DATASET_PATH, 'final_cross_encoder_model')
SCALER_PATH = os.path.join(MODEL_DATASET_PATH, 'scaler.pkl')
COLS_PATH = os.path.join(MODEL_DATASET_PATH, 'numerical_cols.pkl')
UTILS_PATH = os.path.join(MODEL_DATASET_PATH, 'feature_utils.pkl')

# 🧠 --- 모델 및 도구 불러오기 ---
print(f"모델을 로컬 경로에서 불러옵니다: {MODEL_PATH}")
final_model = CrossEncoder(MODEL_PATH)
print("모델 로딩 완료.")

# 전처리 도구들 불러오기
scaler = joblib.load(SCALER_PATH)
numerical_cols = pickle.load(open(COLS_PATH, 'rb'))
feature_utils = pickle.load(open(UTILS_PATH, 'rb'))

# 저장했던 함수들을 현재 노트북에서 사용할 수 있도록 변수에 할당
create_features = feature_utils['create_features']
prepare_cross_encoder_input = feature_utils['prepare_cross_encoder_input']
print("전처리 도구 로딩 완료.")

# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 디바이스: {device}")

## 2단계: 테스트 데이터 준비
불러온 함수들을 사용하여 `test.csv` 데이터를 모델이 예측할 수 있는 형태로 변환합니다.

In [ ]:
# 테스트 데이터 로드
test_df = pd.read_csv(TEST_CSV_PATH)

# 1. 학습 때와 동일한 특징 생성
print("테스트 데이터 특징 생성 중...")
test_features_df = create_features(test_df)

# 2. 학습 때 사용했던 스케일러로 수치 특징 변환
print("수치 특징 스케일링 중...")
numerical_features_test = scaler.transform(test_features_df[numerical_cols])

# 3. CrossEncoder 모델 입력 형식으로 변환
print("모델 입력 데이터 준비 중...")
test_inputs = []
for idx, row in tqdm(test_features_df.iterrows(), total=len(test_features_df)):
    ce_input = prepare_cross_encoder_input(
        row['rule'], 
        row['body']
        # 테스트 데이터에는 예시(example) 컬럼이 없으므로 생략
    )
    
    if '[댓글]' in ce_input:
        rule_part = ce_input.split('[댓글]')[0].strip()
        comment_part = ce_input.split('[댓글]')[1].strip()
        test_inputs.append([rule_part, comment_part])
    else:
        parts = ce_input.split()
        mid = len(parts) // 2
        rule_part = ' '.join(parts[:mid])
        comment_part = ' '.join(parts[mid:])
        test_inputs.append([rule_part, comment_part])
        
print("데이터 준비 완료.")

## 3단계: 예측 실행 및 제출 파일 생성
준비된 데이터를 모델에 입력하여 예측을 수행하고, 결과를 `submission.csv` 파일로 저장합니다.

In [ ]:
# 🚀 --- 예측 실행 ---
print("모델 예측 시작...")
# GPU 메모리 상황에 따라 batch_size를 조절할 수 있습니다. (예: 16, 32, 64)
predictions = final_model.predict(test_inputs, show_progress_bar=True, batch_size=32)

# CrossEncoder의 출력은 로짓(logit)이므로, 확률로 변환하기 위해 시그모이드 함수 적용
probabilities = 1 / (1 + np.exp(-predictions))
print("예측 완료.")

# 📄 --- 제출 파일 생성 ---
submission_df = pd.DataFrame({
    'row_id': test_df['row_id'],
    'rule_violation': probabilities
})

submission_df.to_csv('submission.csv', index=False)

print("\n✅ submission.csv 파일 생성이 완료되었습니다!")
print("제출 파일 샘플:")
display(submission_df.head())